# EDA and Modeling\nThis notebook walks through EDA, preprocessing, training and SHAP explanations. Run the scripts in `src/` for full training.

In [ ]:
import pandas as pd\npd.set_option('display.max_columns', None)\ndf = pd.read_csv('data/processed/players_processed.csv')\ndf.describe()